# Inicialização do DataFrame

In [ ]:
#Transformando o DataFrames em uma tabela temporária de consultas
df_movies.createOrReplaceTempView("movies")

# Comando SQL para respostas da perguntas de negócio

## Quantos filmes estão disponíveis na Amazon? E na Netflix?

In [ ]:
#Amazon
qtd_filmes_amazon = spark.sql("SELECT count(DISTINCT title) as qtd_filmes_amazon\
                               FROM movies\
                               WHERE company == 'amazon'")
qtd_filmes_amazon.show()
#Resposta obtida: 328.205

In [ ]:
#Netflix
qtd_filmes_netflix = spark.sql("SELECT count(DISTINCT title) as qtd_filmes_netflix\
                                FROM movies\
                                WHERE company == 'netflix'")
qtd_filmes_netflix.show()
#Resposta obtida: 17.240

## Dos filmes disponíveis na Amazon, quantos % estão disponíveis na Netflix?

In [ ]:
#Esse desafio de negócios tentei resolver de duas formas diferentes, vou explicitar as duas para mostrar meu raciocínio.
#Na primeira abaixo o resultado obtido foi nulo, então para resolver o problema trago a segunda opção logo em seguida

porcen_filmes_amazon_and_netflix = spark.sql("SELECT CAST((SELECT COUNT(*)\
	                                                         FROM (SELECT * FROM movies WHERE company == 'amazon') a \
	                                                         INNER JOIN (SELECT * FROM movies WHERE company == 'netflix') n \
	                                                         ON a.title = n.title) AS DECIMAL(4, 2)) / \
                                                     CAST(COUNT(*) AS DECIMAL(4, 2))\
                                              AS porcen_filmes_amazon_e_netflix \
                                              FROM movies \
                                              WHERE company == 'amazon'")
porcen_filmes_amazon_and_netflix.show()
#Resposta obtida: Null

In [ ]:
#Método funcional para determinação da porcentagem
#Calculando a quantidade de filmes que estão em ambas as plataformas
qtd_filmes_amazon_and_netflix = spark.sql("SELECT COUNT(a.title)\
	                                         FROM (SELECT DISTINCT title FROM movies WHERE company == 'amazon') a \
	                                         INNER JOIN (SELECT DISTINCT title FROM movies WHERE company == 'netflix') n \
	                                         ON a.title = n.title)")
qtd_filmes_amazon_and_netflix = qtd_filmes_amazon_and_netflix.collect()[0][0]

#Calculando a quantidade de filmes da Amazon
qtd_filmes_amazon = spark.sql("SELECT count(DISTINCT title) as qtd_filmes_amazon\
                               FROM movies\
                               WHERE company == 'amazon'")
qtd_filmes_amazon = qtd_filmes_amazon.collect()[0][0]

percent_amazon_and_netflix = (qtd_filmes_amazon_and_netflix / qtd_filmes_amazon)*100
print(percent_amazon_and_netflix)

#Resposta obtida: 4,2677%

## O quão perto a média das notas dos filmes disponíveis na Amazon está dos filmes disponíveis na Netflix?

In [ ]:
#Proximidade das médias de notas
prox_rating_filmes_amazon_and_netflix = spark.sql("SELECT CAST(\
                                                          ((SELECT AVG(rating)\
                                                            FROM movies\
                                                            WHERE company == 'amazon') - \
                                                           (SELECT AVG(rating) \
                                                            FROM movies \
                                                            WHERE company == 'netflix')) \
                                                          AS DECIMAL(4, 2) \
                                                  ) AS prox_rating_filmes_amazon_and_netflix \
                                                  FROM movies \
                                                  LIMIT 1")
prox_rating_filmes_amazon_and_netflix.show()
#Resposta obtida:0,65

## Qual ano de lançamento possui mais filmes na Netflix?

In [ ]:
#Ano de lançamento com mais filmes
year_max_release = spark.sql("SELECT year_release, count(title) as year_max_release\
                              FROM movies\
                              WHERE company == 'netflix'\
                              GROUP BY year_release\
                              ORDER BY count(title) DESC LIMIT 1")
year_max_release.show()
#Resposta obtida:2004 com 1435 lançados

## Quais filmes que não estão disponíveis no catálogo da Amazon foram *melhor avaliados? *Melhores notas são as 4 e 5

In [ ]:
#Filmes melhor avaliados não disponíveis na Amazon
movies_netflix_not_amazon = spark.sql("SELECT DISTINCT n.title, n.rating\
                                       FROM (SELECT DISTINCT title, rating, company FROM movies WHERE company == 'netflix') n\
                                       LEFT JOIN (SELECT title, rating, company FROM movies WHERE company == 'amazon') a\
                                       ON n.title = a.title\
                                       WHERE a.title IS NULL AND n.rating >= 4\
                                       ORDER BY n.rating DESC")

movies_netflix_not_amazon.show(truncate=False)
#Resposta obtida:São 3233 títulos

## Quais filmes que não estão disponíveis no catálogo da Netflix foram *melhor avaliados? *Melhores notas são as 4 e 5

In [ ]:
#Filmes melhor avaliados não disponíveis na Netflix
movies_amazon_not_netflix = spark.sql("SELECT a.title, AVG(a.rating)\
                                       FROM (SELECT title, rating, company FROM movies WHERE company == 'amazon') a\
                                       LEFT JOIN (SELECT DISTINCT title, rating, company FROM movies WHERE company == 'netflix') n\
                                       ON a.title = n.title\
                                       WHERE n.title IS NULL and a.rating >= 4\
                                       GROUP BY a.title\
                                       ORDER BY AVG(a.rating) DESC")
movies_amazon_not_netflix.show(truncate=False)
#Resposta obtida: 277.443

## Quais os 10 filmes que possuem mais avaliações nas plataformas?

In [ ]:
#Amazon
top_10_movies_amazon = spark.sql("SELECT title, count(title) as top10_movies_amazon\
                                  FROM movies\
                                  WHERE company == 'amazon'\
                                  GROUP BY title\
                                  ORDER BY count(title) DESC LIMIT 10")
top_10_movies_amazon.show(truncate=False)
#Resposta obtida:
'''
+----------------+-------------------+
|           title|top10_movies_amazon|
+----------------+-------------------+
|           pilot|              58965|
|         bosch 1|              54216|
| downton abbey 1|              23811|
| downton abbey 3|              23785|
|   transparent 1|              21340|
|     justified 1|              18859|
|          frozen|              16404|
|under the dome 1|              15734|
| downton abbey 4|              15302|
| downton abbey 2|              14215|
+----------------+-------------------+
'''

In [ ]:
#Netflix
top_10_movies_netflix = spark.sql("SELECT title, count(title) as top10_movies_netflix\
                                   FROM movies\
                                   WHERE company == 'netflix'\
                                   GROUP BY title\
                                   ORDER BY count(title) DESC LIMIT 10")
top_10_movies_netflix.show(truncate=False)
#Resposta obtida:
'''
+--------------------+--------------------+
|               title|top10_movies_netflix|
+--------------------+--------------------+
|   miss congeniality|              232944|
|    independence day|              216596|
|         the patriot|              211764|
|       the godfather|              206551|
|the day after tom...|              196397|
|pirates of the ca...|              193941|
|        pretty woman|              193295|
|             twister|              187808|
|  gone in 60 seconds|              183590|
|        forrest gump|              181508|
+--------------------+--------------------+
'''


## Quais são os 5 clientes que mais avaliaram filmes na Amazon e quantos produtos diferentes eles avaliaram? E na Netflix?

In [ ]:
#Amazon
top_5_customers_amazon = spark.sql("SELECT customer_id, count(DISTINCT title) as top_5_customers_amazon\
                                    FROM movies\
                                    WHERE company == 'amazon'\
                                    GROUP BY customer_id\
                                    ORDER BY count(title) DESC LIMIT 5")
top_5_customers_amazon.show()
#Resposta obtida:
'''
+-----------+----------------------+
|customer_id|top_5_customers_amazon|
+-----------+----------------------+
|   43430756|                  6381|
|   18116317|                  3992|
|   52287429|                  2978|
|   52496677|                  2895|
|   20018062|                  2786|
+-----------+----------------------+
'''

In [ ]:
#Netflix
top_5_customers_netflix = spark.sql("SELECT customer_id, count(DISTINCT title) as top_5_customers_netflix\
                                     FROM movies\
                                     WHERE company == 'netflix'\
                                     GROUP BY customer_id\
                                     ORDER BY count(title) DESC LIMIT 5")
top_5_customers_netflix.show()
#Resposta obtida:
'''
+-----------+-----------------------+
|customer_id|top_5_customers_netflix|
+-----------+-----------------------+
|     305344|                  17129|
|     387418|                  16919|
|    2439493|                  16087|
|    1664010|                  15380|
|    2118461|                  14404|
+-----------+-----------------------+
''

## Quantos filmes foram avaliados na data de avaliação mais recente na Amazon?

In [ ]:
#Amazon
qtd_movies_recent_rating_amazon = spark.sql("SELECT count(DISTINCT title) as qtd_best_rating_amazon\
                                           FROM movies\
                                           WHERE review_date = (SELECT MAX(review_date) FROM movies) and company == 'amazon'")
qtd_movies_recent_rating_amazon.show()
#Resposta obtida: 17.613

## Quais os 10 filmes mais bem avaliados nesta data?

In [ ]:
top_10_movies_recent_rating_amazon = spark.sql("SELECT DISTINCT title, rating\
                                              FROM movies\
                                              WHERE review_date = (SELECT MAX(review_date) FROM movies) and company == 'amazon'\
                                              ORDER BY rating DESC LIMIT 10")
top_10_movies_recent_rating_amazon.show()
#Resposta obtida:
'''
+------------------+--------------------------------+
|title             |top_10_movies_best_rating_amazon|
+------------------+--------------------------------+
|digging for fire  |5                               |
|mad dogs 1        |5                               |
|hotel transylvania|5                               |
|downton abbey 3   |5                               |
|the good wife 3   |5                               |
|immortals         |5                               |
|shaun the sheep 4 |5                               |
|grantchester 1    |5                               |
|the switch        |5                               |
|cloud atlas       |5                               |
+------------------+--------------------------------+
'''

## Quantos filmes foram avaliados na data de avaliação mais recente na Netflix?

In [ ]:
#Netflix
qtd_movies_recent_rating_netflix = spark.sql("SELECT count(DISTINCT title) as qtd_best_rating_netflix\
                                            FROM movies\
                                            WHERE review_date = (SELECT MAX(review_date) FROM movies) and company == 'netflix'")
qtd_movies_recent_rating_netflix.show()
#Resposta obtida:6838

## Quais os 10 filmes mais bem avaliados nesta data?

In [ ]:
top_10_movies_best_rating_netflix = spark.sql("SELECT DISTINCT title, rating\
                                               FROM movies\
                                               WHERE review_date = (SELECT MAX(review_date) FROM movies) and company == 'netflix'\
                                               ORDER BY rating DESC LIMIT 10")
top_10_movies_best_rating_netflix.show(truncate=False)
#Resposta obtida:
'''
+------------------------------+------+
|title                         |rating|
+------------------------------+------+
|the alamo                     |5     |
|ikiru                         |5     |
|the twilight zone vol 39      |5     |
|tom and jerrys greatest chases|5     |
|20                            |5     |
|friends 1                     |5     |
|strictly ballroom             |5     |
|finding neverland             |5     |
|the green mile                |5     |
|the man with the golden gun   |5     |
+------------------------------+------+
'''